# blocs

In [1]:
from functools import reduce
import pandas as pd
import re
import requests

In [2]:
BLOCS = [
    ['vote_type', 'mail'],
    ['vote_type', 'eday'],
    ['pgh_region', 'nside'],
    ['pgh_region', 'somon'],
    ['pgh_region', 'dt_oak_haz'],
    ['pgh_region', 'w14'],
    ['pgh_region', 'rem_ee'],
]

In [3]:
def get_vote_type_pct(vote_type, candidates):
    r = requests.get(
        'https://results.enr.clarityelections.com/PA/Allegheny/109361/277443/json/%s.json' % vote_type,
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
      })
    r = r.json()

    vote_type_sum = [p for p in r['Contests'] if p['A'] == '-1'][0]
    election_vote_type_sum = vote_type_sum['V'][vote_type_sum['C'].index(election_id)]
    return [
        election_vote_type_sum[0]/sum(election_vote_type_sum),
        election_vote_type_sum[1]/sum(election_vote_type_sum),
        sum(election_vote_type_sum[(len(candidates) - 1):])/sum(election_vote_type_sum)
    ]

In [4]:
muni = pd.read_csv('muni.csv')
dup_muni_names = muni[muni.duplicated(subset='name')].name.unique()

muni.loc[
    (muni.name.isin(dup_muni_names)) & (muni.type == 'BOROUGH'),
    'name'
] = muni.name + ' BR'

muni.loc[
    (muni.name.isin(dup_muni_names)) & (muni.type == 'TOWNSHIP'),
    'name'
] = muni.name + ' TP'


### Set rough "regions" based on school district
muni.loc[
    muni.schoold.isin([
        'Avonworth', 'North Allegheny', 'North Hills', 'Northgate', 'Pine Richland', 'Quaker Valley',
        'Upper St. Clair Area', 'Mt. Lebanon', 'Pine-Richland', 'Wilkinsburg'
    ]),
    'schoold'
] = 'Suburban Liberal Core'

muni.loc[muni.schoold.isin(['Gateway', 'Riverview', 'Penn Hills', 'Plum Borough', 'Brentwood Borough', 'Highlands', 'Chartiers Valley']), 'schoold'] = 'ES'
muni.loc[muni.schoold.isin(['Baldwin-Whitehall', 'South Park', 'Steel Valley', 'Cornell', 'Elizabeth Forward']), 'schoold'] = 'SP'
muni.loc[muni.schoold.isin(['Carlynton', 'Bethel Park', 'Keystone Oaks', 'Moon Area', 'Hampton Township', 'Woodland Hills', 'Fox Chapel Area']), 'schoold'] = 'KO BP'
muni.loc[muni.schoold.isin([
    'Allegheny Valley', 'Deer Lakes',
    'Clairton City', 'West Mifflin Area', 'East Allegheny', 'Duquesne City', 'McKeesport Area', 'South Allegheny'
]), 'schoold'] = 'AV/MV'
muni.loc[muni.schoold.isin(['Montour', 'Sto-Rox', 'Fort Cherry', 'Shaler Area']), 'schoold'] = 'W Sub'
muni.loc[muni.schoold.isin(['South Fayete Township', 'West Allegheny', 'West Jefferson Hills', 'Penn-Trafford']), 'schoold'] = 'SW'

In [5]:
PRECINCT_REGEX_PGH = 'PITTSBURGH \w+ (\d+) \w+ \d+'

def format_precinct(precinct):
    precinct = re.sub('CASL SHANNON', 'CASTLE SHANNON', precinct)
    precinct = re.sub("OHARA", "O'HARA", precinct)
    precinct = re.sub('ROSSLYN FARM', 'ROSSLYN FARMS', precinct)
    precinct = re.sub('SPRINGDAL BR', 'SPRINGDALE BR', precinct)
    precinct = re.sub('SPRINGDALE TWP', 'SPRINGDALE TP', precinct)
    precinct = re.sub('UP ST CLAIR', 'UPPER ST. CLAIR', precinct)

    precinct = re.sub(r'^E ', 'EAST ', precinct)
    precinct = re.sub(r'^MT ', 'MOUNT ', precinct)
    precinct = re.sub(r'^N ', 'NORTH ', precinct)
    precinct = re.sub(r'^S ', 'SOUTH ', precinct)
    precinct = re.sub(r'^W ', 'WEST ', precinct)
    precinct = re.sub(' HL', ' HILLS', precinct)
    precinct = re.sub(' HTS', ' HEIGHTS', precinct)
    precinct = re.sub(' HT', ' HEIGHTS', precinct)
    precinct = re.sub(' PK', ' PARK', precinct)
    precinct = re.sub(' VILL', ' VILLAGE', precinct)

    return precinct

def group_non_pgh_precinct(precinct):
    precinct = precinct.replace(' WD', ' WARD')
    precinct = precinct.replace(' WRD', ' WARD')
    precinct = re.sub(r' DIST.*', '', precinct)
    precinct = re.sub(r' WARD.*', '', precinct)
    
    return muni[muni.name == precinct].schoold.values[0]

def group_pgh_precinct(precinct):
    ward = int(re.match(PRECINCT_REGEX_PGH, precinct).group(1))

    if ward == 14:
        return 'w14'
    elif ward in range(1, 7) or ward == 15:
        return 'dt_oak_haz'
    elif ward in range(6, 15):
        return 'rem_ee'
    elif ward in range(16, 21) or ward in range(27, 33):
        return 'somon'
    elif ward in range(21, 27):
        return 'nside'

def get_vote_region_pct(candidates, year, central_election_id, current_version):
    ### Get precinct results
    r = requests.get(
        'https://results.enr.clarityelections.com/PA/Allegheny/%s/%s/json/ALL.json' % (central_election_id, current_version),
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
      })
    r = r.json()

    results = pd.DataFrame(columns=(['precinct'] + candidates))
    for precinct in r['Contests']:
        if election_id in precinct['C']:
            precinct_results = precinct['V'][precinct['C'].index(election_id)]
            precinct_results = precinct_results[0:(len(candidates) - 1)] + [sum(precinct_results[(len(candidates) - 1):])]
            results.loc[len(results.index)] = [precinct['A']] + precinct_results

    results = results.head(-1)
    
    ### Group by voting region
    results['precinct'] = results.precinct.apply(format_precinct)
    results['type'] = ''

    results.loc[~results.precinct.str.startswith('PITTSBURGH'),'class'] = 'alle_region'
    results.loc[
        ~results.precinct.str.startswith('PITTSBURGH'),
        'type'] = results.precinct.apply(group_non_pgh_precinct)

    results.loc[results.precinct.str.startswith('PITTSBURGH'),'class'] = 'pgh_region'
    results.loc[
        results.precinct.str.startswith('PITTSBURGH'),
        'type'] = results[results.precinct.str.startswith('PITTSBURGH')].precinct.apply(group_pgh_precinct)

    ### Sum by voting region, calculate candidate percentage
    results = pd.pivot_table(results, index=['class', 'type'], values=candidates, aggfunc='sum').reset_index()

    results['total'] = results.sum(axis=1)
    for candidate in candidates:
        results['pct_%s%s' % (candidate[0:4].lower(), year)] = results[candidate] / results.total

    return results    

### 2019 ACC at-large primary

In [6]:
### Set candidate names, election ID
candidates = ['Hallam', 'DeFazio', 'Other']
election_id = '0113'

acc19 = pd.DataFrame(columns=['class', 'type'])
acc19['pct_hall19'] = ''
acc19['pct_defa19'] = ''
acc19['pct_othe19'] = ''

In [7]:
### Zero-count the vote-type breakdown (VBM not available)
acc19.loc[len(acc19.index)] = ['vote_type', 'mail'] + [0, 0, 0]
acc19.loc[len(acc19.index)] = ['vote_type', 'eday'] + [0, 0, 0]

In [8]:
### Get precinct results
voting_region_results = get_vote_region_pct(candidates, '19', '95692', '228690')
acc19 = pd.concat([acc19, voting_region_results[acc19.columns]])

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_65915/3744598812.py:81: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results['total'] = results.sum(axis=1)


### 2021 Pittsburgh mayoral primary

In [9]:
### Set candidate names, election ID
candidates = ['Gainey', 'Peduto', 'Other']
election_id = '0032'

pghmayoral21 = pd.DataFrame(columns=['class', 'type'])
pghmayoral21['pct_gain21'] = ''
pghmayoral21['pct_pedu21'] = ''
pghmayoral21['pct_othe21'] = ''

In [10]:
### Get mail, eday results
pghmayoral21.loc[len(pghmayoral21.index)] = ['vote_type', 'mail'] + get_vote_type_pct('Absentee', candidates)
pghmayoral21.loc[len(pghmayoral21.index)] = ['vote_type', 'eday'] + get_vote_type_pct('Election_Day', candidates)

In [11]:
### Get precinct results
voting_region_results = get_vote_region_pct(candidates, '21', '109361', '277443')
pghmayoral21 = pd.concat([pghmayoral21, voting_region_results[pghmayoral21.columns]])

/var/folders/m6/nmg96kbd65n3tz43pzrdnnw00000gn/T/ipykernel_65915/3744598812.py:81: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  results['total'] = results.sum(axis=1)


### Results

In [12]:
dfs = [
    pd.DataFrame(BLOCS, columns=['class', 'type']),
    pghmayoral21,
    acc19
]

df = reduce(lambda x, y: pd.merge(x,y, on=['class', 'type'], how='outer'), dfs)
df

,class,type,pct_gain21,pct_pedu21,pct_othe21,pct_hall19,pct_defa19,pct_othe19
0,vote_type,mail,0.357575,0.526308,0.116116,0.000000,0.000000,0.000000
1,vote_type,eday,0.525235,0.312933,0.161832,0.000000,0.000000,0.000000
2,pgh_region,nside,0.483487,0.368371,0.148141,0.617364,0.379242,0.003394
3,pgh_region,somon,0.305441,0.407443,0.287116,0.517342,0.479106,0.003552
4,pgh_region,dt_oak_haz,0.509474,0.385675,0.104851,0.600179,0.396694,0.003127
5,pgh_region,w14,0.441891,0.518149,0.039959,0.752229,0.245672,0.002098
6,pgh_region,rem_ee,0.617377,0.311485,0.071138,0.619896,0.376373,0.003731
7,alle_region,AV/MV,NaN,NaN,NaN,0.380877,0.616053,0.003071
8,alle_region,City of Pittsburgh,NaN,NaN,NaN,0.460317,0.539683,0.000000
9,alle_region,ES,NaN,NaN,NaN,0.454223,0.542452,0.003325
